# NER and Network-of-Terms pipeline

Basic experiment for creating an NER and keyword extraction pipeline in combination with querying the [NDE Netwerk of Terms](https://github.com/netwerk-digitaal-erfgoed/network-of-terms). The code inspired by the Hands-On 1.2 code from the [Open HPI Knowledge Graph Course 2023](https://open.hpi.de/courses/knowledgegraphs2023).

This colab contains the (slightly adjusted) code from the [ner-not-pipeline](https://github.com/EnnoMeijers/ner-not-pipeline) repo.

We'll start with downloading and importing required libs

In [ ]:
!pip install python_graphql_client
!python -m spacy download nl_core_news_lg

import spacy
import json
import sys
from spacy.matcher import Matcher
from python_graphql_client import GraphqlClient

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.1/568.1 MB 1.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('nl_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Define the sentence to perform the NER on (uncomment to use other examples or define your own text).

In [ ]:
# based on https://github.com/EnnoMeijers/ner-not-pipeline/blob/main/stadhuis-veere.txt
sentence = "Het stadhuis van Veere. Laatgotisch gebouw, in 1474 begonnen. Waarschijnlijk ontworpen door mr Evert Spoorwater, geboren te Gouda. De gevel van gobertangersteen is nog vrijwel geheel in de oorspronkelijke toestand. Zij vertoont uitgekraagde hoektorentjes, stenen kruiskozijnen overdekt met korfbogen, waarin verdiepte velden versierd met gotische traceringen, en nissen in de vensterdammen op de verdieping. De oorspronkelijke beelden, in 1517 vervaardigd door Michiel Ywijns, zijn uit de nissen verwijderd en vervangen door replieken van de hand van O. Wenckebach uit Amersfoort, welke heren en vrouwen van Veere voorstellen. Een bordes uit 1588, in 1749 gewijzigd in Lodewijk XIV stijl, geeft toegang tot de hoofdingang, terwijl in het travee rechts van de ingangspartij het venster geflankeerd wordt door twee beeldnissen. De traptoren, aangebouwd tegen de bakstenen achtergevel en behorend bij het oorspronkelijke plan, werd na 1591 gedeeltelijk afgebroken en tussen 1594 en 1599 vervangen door een slanke bovenbouw, die ontworpen is door Adriaen de Muer, geboortig uit Brugge."

# based on https://github.com/EnnoMeijers/ner-not-pipeline/blob/main/haarlem.txt
# sentence = "Haarlem behoort tot de middelgrote steden in de Randstad. Tot de gemeente Haarlem behoren de stad Haarlem en het westelijke deel van het dorp Spaarndam. Haarlem telt 165.650 inwoners[1] en is daarmee na Amsterdam de grootste stad van Noord-Holland en de dertiende gemeente van Nederland. De grootstedelijke agglomeratie Haarlem (Haarlem, Heemstede, Bloemendaal en Zandvoort) telt ongeveer 235.000 inwoners,[1] en het stadsgewest Haarlem (Zuid-Kennemerland en IJmond) ruim 385.000 inwoners.[1] Haarlem wordt voor het eerst genoemd in een document uit de 10e eeuw. In 1245 kreeg het stadsrechten van Willem II van Holland. Aan het eind van de middeleeuwen was Haarlem een van de belangrijkste steden van Holland geworden. In de Vroegmoderne Tijd ontwikkelde de stad zich op industrieel gebied als textielstad en op cultureel gebied als schildersstad."

# based on https://github.com/EnnoMeijers/ner-not-pipeline/blob/main/netsuke.txt
# sentence = "Netsuke - Japanse gordelknoop. Omdat een traditionele Japanse kimono geen zakken had, werd alles aan koorden (himo) gehangen. De koorden werden achter een riem (obi) doorgehaald. Om ervoor te zorgen dat de koorden bleven hangen, werden ze aan de bovenkant met een netsuke (gordelknoop) vastgemaakt. De voorstelling van de netsuke had een diepere betekenis. Meestal droeg de drager er een van zijn geboortejaar, bijvoorbeeld het jaar van de aap. Het materiaal kon van alles zijn. Zo zal een houthakker meestal hout gebruiken, een visser zal een netsuke van visbeen maken. Deze netsuke is van porselein en komt uit de collectie gemeente Vlissingen."



Next, we defined the terminology source to be searched via the Network of Terms, per NER resultype.

In [ ]:
# based on https://github.com/EnnoMeijers/ner-not-pipeline/blob/main/config.json

config={'CONCEPT': [
                    'http://vocab.getty.edu/aat/sparql',
                    'https://data.cultureelerfgoed.nl/PoolParty/sparql/term/id/cht'
                   ],
        'DATE': [
                    'http://vocab.getty.edu/aat/sparql/styles-and-periods',
                    'https://data.cultureelerfgoed.nl/PoolParty/sparql/term/id/cht/styles-and-periods'
                ],
        'EVENT': [
                    'https://query.wikidata.org/sparql#entities-all'
                 ],
        'GPE': [
                    'https://demo.netwerkdigitaalerfgoed.nl/geonames',
                    'https://query.wikidata.org/sparql#entities-places'
               ],
        'ORG': [
                    'https://query.wikidata.org/sparql#entities-all'
               ],
        'PERSON': [
                    'https://data.netwerkdigitaalerfgoed.nl/rkd/rkdartists/sparql',
                    'http://data.bibliotheken.nl/thesp/sparql',
                    'https://data.muziekschatten.nl/sparql/#personen',
                    'https://query.wikidata.org/sparql#entities-persons',
                    'https://data.beeldengeluid.nl/id/datadownload/0030'
                  ]
        }


Some function definitions...

In [ ]:
def queryTN(sources,searchTerm):

    # Prepare the search query
    query = """
      query tn($sources: [ID]!, $searchTerm: String!) {
        terms( sources: $sources, query: $searchTerm ) {
          result {
            __typename
            ... on Terms {
              terms { uri prefLabel altLabel hiddenLabel scopeNote seeAlso }
            }
            ... on Error {
              message
            }
          }
        }
      }
    """

    # Perform a synchronous request for simplicity
    return client.execute(query=query, variables= {"sources": sources, "searchTerm": searchTerm })

def matchLabel(labels,searchLabel):
  for label in labels:
    if label.strip().lower() == searchLabel:
      return label
  return False

def Refine(ner,nerType):

  # only proces nerTypes that are defined in the config file
  if not (nerType in config):
    return False

  # use source selection from the config.json
  sourceList=config[nerType]

  print("- looking up",nerType,ner,"via",sourceList)
  # perform Network of Terms request for this NER
  data=queryTN(sourceList,ner)

  # select the resultLists per source
  resultList = data['data']['terms']
  for results in resultList:
    if(results['result']['__typename']=="Terms"):
      terms=results['result']['terms']
      for term in terms:
        found=matchLabel(term['prefLabel'],ner)
        if(found):
          return term
        found=matchLabel(term['altLabel'],ner)
        if(found):
          return term
  return False

def processKeywords():
  token_details = []
  print("Processing keywords")
  for token in doc:
    if(token.pos_=="NOUN"):
      if not token.text in termList:
        termFound=Refine(token.text,"CONCEPT")
        if(termFound):
          termList[token.text]=termFound
  print("\nKeywords processing finshed!")

def processNERs():
  ner_details = []
  for ent in doc.ents:
    row=(ent.text, ent.label_,spacy.explain(ent.label_))
    if not (row in ner_details):
      ner_details.append(row)

  print("Processing named entities")
  for row in ner_details:
    ner=row[0].strip().lower()
    nerType=row[1]
    if not ner in termList:
      termFound=Refine(ner,nerType)
      if(termFound):
        termList[ner]=termFound
  print("\nNER processing finished!")

def writeCSV():
  outFile='results.csv'
  with open(outFile,"w") as fileHandle:
    print('searchTerm;URI;prefLabel;altLabel;scopeNote',file=fileHandle)
    for term in termList:
      print(
        term +";"+
        termList[term]['uri'] +";"+
        ', '.join(termList[term]['prefLabel']) +";"+
        ', '.join(termList[term]['altLabel']) +";"+
        ', '.join(termList[term]['scopeNote']),
        file=fileHandle
      )
  print("Results written to",outFile)

Now it's time to perform the NER and Refine the found terms.

In [ ]:
# Specify the Network-of-Terms GraphQL API
client = GraphqlClient(endpoint="https://termennetwerk-api.netwerkdigitaalerfgoed.nl/graphql")

# load a Dutch language model
nlp = spacy.load("nl_core_news_lg")
doc = nlp(sentence)

# initialize resultlist
termList = {}

# find relevant concepts URIs based on the nouns in the text, commented out to speed things up
# processKeywords()

# find relevant URIs for locations and persons based on the namend entities in the text
processNERs()

Processing named entities
- looking up GPE veere via ['https://demo.netwerkdigitaalerfgoed.nl/geonames', 'https://query.wikidata.org/sparql#entities-places']
- looking up DATE 1474 via ['http://vocab.getty.edu/aat/sparql/styles-and-periods', 'https://data.cultureelerfgoed.nl/PoolParty/sparql/term/id/cht/styles-and-periods']
- looking up PERSON mr evert spoorwater via ['https://data.netwerkdigitaalerfgoed.nl/rkd/rkdartists/sparql', 'http://data.bibliotheken.nl/thesp/sparql', 'https://data.muziekschatten.nl/sparql/#personen', 'https://query.wikidata.org/sparql#entities-persons', 'https://data.beeldengeluid.nl/id/datadownload/0030']
- looking up GPE gouda via ['https://demo.netwerkdigitaalerfgoed.nl/geonames', 'https://query.wikidata.org/sparql#entities-places']
- looking up DATE 1517 via ['http://vocab.getty.edu/aat/sparql/styles-and-periods', 'https://data.cultureelerfgoed.nl/PoolParty/sparql/term/id/cht/styles-and-periods']
- looking up PERSON michiel ywijns via ['https://data.netwerkd

Show results in pretty JSON

In [ ]:
print(json.dumps(termList, indent=4))

# write results to the file results.csv
writeCSV()

{
    "veere": {
        "uri": "https://sws.geonames.org/2745739/",
        "prefLabel": [
            "Veere (NL)"
        ],
        "altLabel": [
            "Veera",
            "Vere",
            "\u0412\u0435\u0435\u0440\u0430",
            "Veere"
        ],
        "hiddenLabel": [],
        "scopeNote": [
            "populated place"
        ],
        "seeAlso": []
    },
    "gouda": {
        "uri": "https://sws.geonames.org/2755419/",
        "prefLabel": [
            "Gemeente Gouda (NL)",
            "Gemeente Gouda"
        ],
        "altLabel": [
            "ghwda",
            "\u0628\u0644\u062f\u064a\u0629 \u063a\u0648\u062f\u0629",
            "\u0628\u0644\u062f\u064a\u0629 \u063a\u0648\u062f\u0627",
            "\u063a\u0648\u062f\u0627",
            "Gemeente Gouda",
            "ghwdt",
            "bldyt ghwda",
            "Gouda",
            "bldyt ghwdt",
            "\u063a\u0648\u062f\u0629"
        ],
        "hiddenLabel": [],
        "scopeNote"